In [1]:
import numpy as np
from particles.state_space_models import StochVol, Bootstrap
from particles import SMC
from particles_cdssm.collectors import NLPD, ObservationPredictiveParticles, ObservationPredictiveMoments

I have developed some new `collectors` that you can feed as inputs to the SMC algorithm, to store a representation of the k-step ahead prediction $Y_{t+k}|Y_{1:t}$ online witha particle filter. To use them, you will need to get the import above from the `particles_cdssm` package to work. Minimum working example of usage is presented below:  

In [ ]:
# Default choice of params: in practice we will change to the ones you obtain from using the PMMH to infer the params:
sv_params = {"mu": -1.02, "rho": 0.9702, "sigma": 0.178}
ssm = StochVol(**sv_params)

# I'm using simulated data here - replace it with the S&P 500 data that you have.
# Include both the training data and the test data at this point:
T = 400
x, y = ssm.simulate(T)

# Create bootstrap Feynman-Kac model from the inputs:
fk = Bootstrap(ssm, data=y)

# Define the 'collector' in the following way: parameter K detmerines the number of steps ahead that we want to predict: 
col = ObservationPredictiveParticles(K=1)

# Here are 2 other examples that may be helpful: have look in particles_cdssm/collectors.py at the docstrings for more details:
# col = ObservationPredictiveMoments(K=1)
# col = NLPD(K=1)

# Call SMC in the following way to store full particle estimates of the 1-step ahead returns:
N=1000 # Number of particles: increase to improve accuracy of the inference.
smc_algo = SMC(fk=fk, N=1000, collect=[col], store_history=True)

smc_algo.run()

In [ ]:
# After running the SMC, you will have seen that you can access the full particle estimtes of the filtering distribution in the following way:

# List of the particles at each time step:
smc_algo.hist.X
# List of the weights at each time step:
smc_algo.hist.wgts
# Or, the following command for the first and second moments:
# smc_algo.summaries.moments

# After runnning the above, you can now access the K-step ahead predictive distribution, via:
predictions = smc_algo.summaries.obs_predictive_particles

# (alternatives for the other 2 predictive collectors):
# pred = smc_algo.summaries.obs_predictive_moments
# pred = smc_algo.summaries.nlpd

We can use NLPD (negative log predictive density) and CRPS (Continuous-Ranked Probability score) to evaluate the predictive uncertainty quantification of the model that we fitted, out of sample. Also, have a look in `particles_cdssm/collectors.py` for more details on what is going on.